In [1]:
from os import path, getcwd
import subprocess

D:\Anaconda3\envs\SAML11-32\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [140]:
def atprogram(
        project_path, device_name="ATSAML11E16A", verbose=0, 
        clean=False, build=True, erase=True, program=True, verify=False, 
        jobs=8, tool="EDBG", interface="SWD", 
        atmel_studio_path="C:\\Program Files (x86)\\Atmel\\Studio\\7.0",
        make_path=None, atprogram_path=None,
        configuration="Debug", device_sn=None,
        make_command=None, atprogram_command=None):
    # NOTE: if multiple devices are connected the serial number needs to be supplied to atprogram using the -s flag.
    # NOTE: verification is known to return 23, also in command line.
    elf_mode = path.splitext(project_path)[1] is ".elf"
    stdout = subprocess.PIPE if verbose >= 0 else None
    stderr = subprocess.STDOUT if verbose >= 1 else None
    if not elf_mode:
        if clean or build:
            make_path = make_path or path.join(atmel_studio_path, "shellutils", "make.exe")
            def make_caller(args):
                res = subprocess.run([make_path] + args.split(), cwd=path.join(project_path, configuration), stdout=stdout, stderr=stderr)
                if verbose: print(res.stdout.decode())
                return res.returncode
        if make_command:
            returncode = make_caller(make_command)
            if returncode or not atprogram_command:
                return returncode
        if clean:
            returncode = make_caller("clean")
            if returncode: return returncode
        if build:
            returncode = make_caller(f"all --jobs {jobs} --output-sync")
            if returncode: return returncode
    if erase or program or verify or atprogram_command:
        atprogram_path = atprogram_path or path.join(atmel_studio_path, "atbackend", "atprogram.exe")
        def atprogram_caller(args):
            res = subprocess.run([atprogram_path] + ["-v"]*(verbose-1) + args.split(), stdout=stdout, stderr=stderr)
            if verbose: print(res.stdout.decode())
            return res.returncode
        elf_file_path = None if not (program or verify) else project_path if elf_mode else \
            path.join(project_path, configuration, path.basename(project_path)) + ".elf"
        atprogram_command = atprogram_command or \
            f"-t {tool} -i {interface} -d {device_name}" + \
            (device_sn is not None) * f" -s {device_sn}" + \
            erase * " chiperase" + \
            program * f" program -f {elf_file_path}" + \
            verify * f" verify -f {elf_file_path}" + \
            (verbose >= 3) * " info"
        returncode = atprogram_caller(atprogram_command)
        if returncode:
            atprogram_caller("exitcode")
            return returncode
    return 0

In [104]:
from os import path, getenv
from subprocess import run, PIPE, STDOUT


def atprogram(project_path, device_name="ATSAML11E16A", verbose=0, clean=False,
              build=True, erase=True, program=True, verify=False, jobs=8,
              tool="EDBG", interface="SWD", atmel_studio_path=path.join(
                  getenv("programfiles(x86)"), "Atmel", "Studio", "7.0"),
              make_path=None, atprogram_path=None, configuration="Debug",
              device_sn=None, make_command=None, atprogram_command=None,
              dry_run=False):
    # NOTE: if multiple devices are connected the serial number needs to be
    #   supplied to atprogram using the -s flag.
    # NOTE: verification is known to return 23, also in command line.
    elf_mode = path.splitext(project_path)[1] is ".elf"
    stdout = PIPE if verbose >= 0 else None
    stderr = STDOUT if verbose >= 1 else None
    if not elf_mode:
        if clean or build:
            make_path = make_path or path.join(
                atmel_studio_path, "shellutils", "make.exe")

            def make_caller(command):
                args = ([make_path] + command.split())
                kwargs = dict(cwd=path.join(
                    project_path, configuration), stdout=stdout, stderr=stderr)
                if dry_run:
                    print(args, kwargs)
                    return 0
                res = run(args, **kwargs)
                if verbose:
                    print(res.stdout.decode())
                return res.returncode
        if make_command:
            returncode = make_caller(make_command)
            if returncode or not atprogram_command:
                return returncode
        if clean:
            returncode = make_caller("clean")
            if returncode:
                return returncode
        if build:
            returncode = make_caller(f"all --jobs {jobs} --output-sync")
            if returncode:
                return returncode
    if erase or program or verify or atprogram_command:
        atprogram_path = atprogram_path or path.join(
            atmel_studio_path, "atbackend", "atprogram.exe")

        def atprogram_caller(command):
            args = ([atprogram_path] + (verbose-1) * ["-v"] + command.split())
            kwargs = dict(stdout=stdout, stderr=stderr)
            if dry_run:
                print(args, kwargs)
                return 0
            res = run(args, **kwargs)
            if verbose:
                print(res.stdout.decode())
            return res.returncode
        elf_file_path = None if not (program or verify) else project_path if elf_mode else \
            path.join(project_path, configuration,
                      path.basename(project_path)) + ".elf"
        atprogram_command = atprogram_command or \
            f"-t {tool} -i {interface} -d {device_name}" + \
            (device_sn is not None) * f" -s {device_sn}" + \
            erase * " chiperase" + \
            program * f" program -f {elf_file_path}" + \
            verify * f" verify -f {elf_file_path}" + \
            (verbose >= 3) * " info"
        returncode = atprogram_caller(atprogram_command)
        if returncode:
            atprogram_caller("exitcode")
            return returncode
    return 0


In [105]:
command

NameError: name 'command' is not defined

In [106]:
project_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\pydgilib\\UnitTest\\UnitTest"

In [108]:
atprogram(project_path, verbose=1, dry_run=False)

make: Nothing to be done for 'all'.

Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

In [83]:
from os import getenv
getenv("NUMBER_OF_PROCESSORS")
#!echo %NUMBER_OF_PROCESSORS%

'8'

In [150]:
def g(b=getenv("programfiles(x86)")):
    print(b)
g()

C:\Program Files (x86)


In [73]:
def j(*args):
    print(*args)
    print(args)
    print(type(args))

In [110]:
a = b = c = None
if a is b is None:
    print(4)

4


In [111]:
s = "-v"
def i(e):
    return "v" in e

i(s)

True

In [117]:
assert i("-eee")

AssertionError: 

In [126]:
not True and (True or True)

False

In [22]:
command = "-v 6"
args = (["make.exe"] + command.split())
args
"".join(*args)

TypeError: join() takes exactly one argument (3 given)

In [3]:
makefile_path = path.join(getcwd(),"hello")

In [4]:
def foo():
    foo.counter += 1
    print("Counter is %d" % foo.counter)
foo.counter = 0

In [14]:
kwargs = dict(cwd=makefile_path)

In [18]:
def run(*args, **kwargs):
    run.dry_run_result.append("".join(kwargs.get("cwd", getcwd())) + "> " + " ".join(*args))
run.dry_run_result = []

In [19]:
run(args, **kwargs)
run.dry_run_result

['C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\test notebooks\\hello> make.exe -v 6']

In [162]:
" ".join(args)

'make.exe -v 6'

In [181]:
bytes("sdf", "utf-8").decode()

'sdf'

In [187]:
a = list
a.setattr(a,4)
a.b=3

AttributeError: type object 'list' has no attribute 'setattr'

In [193]:
res = object()
res.stdout = lambda: None

TypeError: object() takes no parameters

In [41]:
verbose = 2
[] + ["-v"]*(verbose-1)

['-v']

In [44]:
device_sn = "adfaf"
f"-s {device_sn}"*(device_sn is not None)

'-s adfaf'

In [34]:
def g(h):
    h or return 4
    return 3

SyntaxError: invalid syntax (<ipython-input-34-e10f4a9a9d09>, line 2)

In [134]:
False or not None

True

In [115]:
a = 4
make_path = None
make_path = make_path or a
cc = None
dd = 8
def g(a):
    dd = dd or 4
    a += bool(cc)
    make_path = make_path or a
    return make_path
g(a)

UnboundLocalError: local variable 'dd' referenced before assignment

In [100]:
make_path = make_path or 5

In [32]:
make_path

4

In [132]:
A = 243
if not True or A:
    print(5)

5


In [81]:
elf_file_path = 4 if not (program or verify) else 5 if elf_mode else \
    6
elf_file_path

6

In [79]:
program = True
verify = False
project_path = 5
elf_mode = False

In [82]:
elf_file_path = None if not (program or verify) else project_path if elf_mode else \
    ".elf"

```
>atprogram
[ERROR] No command specified.
Atmel Studio Command Line Interface
Copyright (C) 2015 Atmel Corporation.

Usage: atprogram [options] <command> [arguments] [<command> [arguments] ...]

Options:
  -t  --tool <arg>           Tool name: avrdragon, avrispmk2, avrone, jtagice3,
                             jtagicemkii, qt600, stk500, stk600, samice, edbg,
                             medbg, nedbg, atmelice, pickit4, powerdebugger,
                             megadfu or flip.
  -s  --serialnumber <arg>   The programmer/debugger serialnumber. Must be
                             specified when more than one debugger is
                             connected.
  -c  --comport <arg>        The com port to use for the programmer/debugger.
                             e.g. -c COM1 or -c 1
  -i  --interface <arg>      Physical interface: aWire, debugWIRE, HVPP, HVSP,
                             ISP, JTAG, PDI, UPDI, TPI or SWD.
  -d  --device <arg>         Device name. E.g. atxmega128a1 or at32uc3a0256.
  -v  --verbose              Verbose output (debug).
  -h  --host <arg>           Target host which runs the backend process.
  -p  --port <arg>           Specify which port to use for the backend process.
                             Ignored if the host option is not given.
  -l  --logfile <arg>        Log file for atpbackend output.
  -f  --force                Force command even if firmware is not up to date.
  -cl --clock <arg>          The frequency to use for communication with a
                             device (hz, khz, mhz, default hz). E.g. -cl 10mhz
  -mb --max-baudrate <arg>   The maximum baud rate for communication on awire
                             (bps).
  -xr --externalreset        Apply external reset when starting a session.
  -dc --daisychain <args,..> Set up a JTAG daisychain. Arguments are <devices-
                             before, devices-after, instr-before, instr-after>.
  -dca --daisychain_auto <arg> Deprecated.
  -sa --snippet_addr <arg>   Address where flash programming ram snippet should
                             be loaded.
  -nr --no_ram               Do not use ram snippet to program flash.
  -tv --target-voltage <arg> Set the STK600, STK500 or Powerdebugger target
                             voltage (float value).
  -a0 --aref0 <arg>          Set the STK600 Aref0 or STK500 aref generator
                             voltage (float value).
  -a1 --aref1 <arg>          Set the STK600 Aref1 generator voltage (float
                             value).
  -cg --clock-generator <arg> Set the STK600 or STK500 clock generator frequency
                             (hz, khz, mhz, default hz).
      --timeout <arg>        Set the timeout value in seconds for commands. The
                             default is 180 seconds. Set to 0 for no timeout.
  -q  --quiet                Do not display activity indicator.
  -?  --help                 Display help information.
  -V  --version              Display version information.
  -cf --configfile <arg>     Set the configuration setting file name with path.
  -ua --12v-updi-activation <arg> Use the 12V activation sequence for UPDI.
                             Disconnect any peripherals on the UPDI line which
                             are not 12V tolerant! Supported modes are: tool-
                             toggle-power, user-toggle-power, simple-unsafe-
                             pulse
  -ab --atbackend <arg>      Specify path to atbackend executable. If specified
                             with --host and --port, then --host and --port is
                             used.

Commands:
   calibrate        Performs the oscillator calibration procedure.
   chiperase        Full erase of chip.
   debugaccess      Set(/get) Debug Access Level (DAL) on ARM devices.
   detect           Detects devices on the jtag interface for supported tools.
   dwdisable        Disable debugWIRE interface.
   erase            Erase the specified memory.
   exitcodes        Display possible exit codes for atprogram.
   help             Displays help for a specific command.
   info             Display information about a device.
   interactive      Run in interactive mode.
   list             Detect and print information about connected Atmel Tools.
   migration        Display help for migration from old command line utilities.
   panel            Pops-up Tool's settings Dialog.
   parameters       Sets given parameters on the given tool without doing anything else.
   powercalibration Performs the calibration procedure for the Powerdebugger.
   program          Program device with data from <file>.
   read             Read the contents of the memory on the device.
   reboot           Reboots the given tool.
   reset            Reset all domains and jump to the reset vector.
   secure           Set the security bit on UC3 and ARM devices.
   selftest         Performs the selftest procedure on Atmel-ICE.
   verify           Verify content of memory based on a file.
   version          Display the version.
   write            Write to the memory with values entered on the command line.

Arguments:
  Use atprogram help <command> to get available attributes.

Example:
  atprogram -t jtagice3 -i jtag -d at32uc3b0512 program -f e:\file.elf
  atprogram -t avrone -i pdi -d atxmega128a1 chiperase

For command specific help, use atprogram help <command>

```

```
>atprogram -d ATSAML11E16A -t EDBG -i SWD exitcode
Exit codes for atprogram

atprogram is the Atmel Studio Command Line Interface which is deployed with
Atmel Studio 6 as well as in a standalone package.  It replaces the individual
Tools' command line interfaces, including:
  - Stk500.exe
  - Stk600.exe
  - jtagiceii.exe
  - AVRDragon.exe
  - avrone.exe

atprogram also replaces the command line interface to UC3 devices- avr32program
atprogram also provides command line access to JTAGICE3 and SAMICE.

atprogram makes use of the Atmel Studio backend, which means that all tools
and devices supported by Atmel Studio are supported by atprogram.exe

This guide is intended to be used as a reference when using atprogram
in script utilities.

Code ranges:
 0-9         Exit codes that is locked or unknown
 10-50       Exit codes due to errors in atprogram itself
 100-200     Exit codes from atbackend


Exits Codes:

0    Success                       The program executed with no errors
1    Unexpected Error              A run time error that was not anticipated
2    User Interrupt Error          A user interrupted the program
10   Command Error                 Exception which indicates that a command has
                                   failed for some internal reason.
11   Command Argument Error        Raised when there are error in the arguments
                                   passed to the command
12   Missing Command Error         An error which occurs if there is no command
13   Command Parser Error          Error when parsing command line arguments.
14   Tool Error                    Exception raised when there is an error with
                                   the setup or use of a tool
15   Resource Error                Exception raised when there is an error in
                                   creating or serving a resource.
16   Tcf Error                     A TCF command returned an error.
17   Timeout Error                 Error because of an timeout.
18   Missing Or Wrong Input Error  An error caused by incomplete information
                                   specified by user
19   Old Fw Exception              Firmware must be upgraded
20   Backend Creation Error        Exception raised when there is an error in
                                   creating or serving a resource.
21   Options Parse Error           Error in parsing an option.
22   Json Parse Error              Exception which can be thrown if a TCF
                                   response could not be parsed.
23   Verify Error                  An error given when failing to verify memory
                                   content
```

In [ ]:
"""This module provides Python bindings for make and atprogram."""
from os import path, getenv
from subprocess import run, PIPE, STDOUT
from warnings import warn
from collections import namedtuple

# In secure mode all commands will be returned in a list, they will not be
# executed
SECURE_MODE = False


def atprogram(project_path=None, device_name="ATSAML11E16A", verbose=0,
              clean=False, build=True, erase=True, program=True, verify=False,
              tool="EDBG", interface="SWD", atmel_studio_path=path.join(
                  getenv("programfiles(x86)"), "Atmel", "Studio", "7.0"),
              make_path=None, atprogram_path=None, configuration="Debug",
              device_sn=None, jobs=getenv("NUMBER_OF_PROCESSORS"),
              make_command=None, atprogram_command=None, dry_run=False):
    """Atprogram.

    This function can compile projects and write them to a device. It
    determines what to do based on the arguments it gets. Specify at least one
    of `project_path`, `make_command` or `atprogram_command`.

    In secure mode all commands will be returned in a list, they will not be
    executed.

    NOTE: Verification is known to return 23, also in command line.

    Keyword Arguments:
        project_path {str} -- Location where the project resides. If it ends in
            `.elf` the elf file will be used. If it ends in `Makefile` the
            Makefile will be used. Otherwise it should be a path to a folder
            which holds the `Debug` folder. (default: {None})
        device_name {str} -- Device name. E.g. atxmega128a1 or at32uc3a0256.
            (default: {"ATSAML11E16A"})
        verbose {int} -- Verbosity:
            - 0: Silent (default: {0})
            - 1: Info
            - 2: Debug
            - 3: List Commands
        clean {bool} -- Run make clean if True (default: {False})
        build {bool} -- Run make all if True (default: {True})
        erase {bool} -- Run atprogram chiperase if True (default: {True})
        program {bool} -- Run atprogram program if True (default: {True})
        verify {bool} -- Run atprogram verify if True (default: {False})
        tool {str} -- Tool name: avrdragon, avrispmk2, avrone, jtagice3,
            jtagicemkii, qt600, stk500, stk600, samice, edbg, medbg, nedbg,
            atmelice, pickit4, powerdebugger, megadfu or flip. (default:
            {"EDBG"})
        interface {str} -- Physical interface: aWire, debugWIRE, HVPP, HVSP,
            ISP, JTAG, PDI, UPDI, TPI or SWD. (default: {"SWD"})
        atmel_studio_path {[type]} -- Location where Atmel Studio is installed,
            ending in the folder named after the version, e.g. 7.0. (default:
            {path.join(getenv("programfiles(x86)"), "Atmel", "Studio", "7.0")})
        make_path {[type]} -- Location where `make.exe` is installed. (default:
            {path.join(getenv("programfiles(x86)"), "Atmel", "Studio", "7.0", "shellutils", "make.exe")})
        atprogram_path {[type]} -- Location where `atprogram.exe` is installed
            (default: {path.join(getenv("programfiles(x86)"), "Atmel", "Studio", "7.0", "atbackend", "atprogram.exe")})
        configuration {str} -- Which configuration to use. (default: {"Debug"})
        device_sn {str} -- The programmer/debugger serialnumber. Must be
            specified when more than one debugger is connected. (default:
            {None})
        jobs {int} -- How many jobs *make* should use(default:
            {getenv("NUMBER_OF_PROCESSORS")})
        make_command {str} -- Options to pass to make `[options] [target] ...`
            (default: {None})
        atprogram_command {str} -- Command(s) to pass to atprogram: `[options]
            <command> [arguments] [<command> [arguments] ...]` (default:
            {None})
        dry_run {bool} -- [description] (default: {False})

    Raises:
        ValueError -- Need to specify at least one of `project_path`,
            `make_command` or `atprogram_command`.

    Returns:
        int -- A non-zero return value indicates the subprocess call returned
            an error.

    """
    if SECURE_MODE:
        warn("In secure mode all commands will be returned in a list, they " +
             "will not be executed.")
    elf_mode = False
    makefile_mode = False
    makefile_path = None
    if dry_run or SECURE_MODE:
        run = _run
        run.dry_run_result = []
    if project_path is not None:
        elf_mode = path.splitext(project_path)[1] is ".elf"
        makefile_path, makefile = path.split(project_path)
        makefile_mode = makefile is "Makefile"
        if not makefile_mode:
            makefile_path = path.join(project_path, configuration)
    elif make_command is atprogram_command is None:
        raise ValueError(
            "Need to specify at least one of project_path, make_command or " +
            "atprogram_command.")
    else:
        # This is make_command mode or atprogram_command mode
        clean = build = erase = program = verify = False
    stdout = PIPE if verbose >= 0 else None
    stderr = STDOUT if verbose >= 1 else None
    if not elf_mode and (clean or build or make_command):
        make_path = make_path or path.join(
            atmel_studio_path, "shellutils", "make.exe")

        def make_caller(command):
            args = ([make_path] + command.split())
            kwargs = dict(cwd=makefile_path, stdout=stdout, stderr=stderr)
            res = run(args, **kwargs)
            if verbose:
                print(res.stdout.decode())
            return res.returncode
        if make_command:
            returncode = make_caller(make_command)
            if returncode or not atprogram_command:
                return returncode
        if clean:
            returncode = make_caller("clean")
            if returncode:
                return returncode
        if build:
            returncode = make_caller(f"all --jobs {jobs} --output-sync")
            if returncode:
                return returncode
    if not makefile_mode and (erase or program or verify or atprogram_command):
        atprogram_path = atprogram_path or path.join(
            atmel_studio_path, "atbackend", "atprogram.exe")

        def atprogram_caller(command):
            args = ([atprogram_path] + (verbose-1) * ["-v"] + command.split())
            kwargs = dict(stdout=stdout, stderr=stderr)
            if dry_run:
                print(args, kwargs)
                return 0
            res = run(args, **kwargs)
            if verbose:
                print(res.stdout.decode())
            return res.returncode
        elf_file_path = None if not (program or verify) else project_path if \
            elf_mode else path.join(project_path, configuration,
                                    path.basename(project_path)) + ".elf"
        atprogram_command = atprogram_command or \
            f"-t {tool} -i {interface} -d {device_name}" + \
            (device_sn is not None) * f" -s {device_sn}" + \
            erase * " chiperase" + \
            program * f" program -f {elf_file_path}" + \
            verify * f" verify -f {elf_file_path}" + \
            (verbose >= 3) * " info"
        returncode = atprogram_caller(atprogram_command)
        if returncode:
            atprogram_caller("exitcode")
            return returncode
    if dry_run or SECURE_MODE:
        if verbose:
            print(run.dry_run_result)
        return run.dry_run_result
    return 0


def _run(*args, **kwargs):
    _run.dry_run_result.append(
        "".join(kwargs.get("cwd")) + "> " + " ".join(*args))
    res = namedtuple("res", ["stdout", "returncode"])
    res.stdout = bytes(_run.dry_run_result[-1], "utf-8")
    res.returncode = 0
    return res


In [ ]:
from os import path, getenv
from subprocess import run, PIPE, STDOUT
from warnings import warn

# In secure mode all commands will be returned in a list, they will not be
# executed
SECURE_MODE = False


def atprogram(project_path=None, device_name="ATSAML11E16A", verbose=0,
              clean=False, build=True, erase=True, program=True, verify=False,
              tool="EDBG", interface="SWD", atmel_studio_path=path.join(
                  getenv("programfiles(x86)"), "Atmel", "Studio", "7.0"),
              make_path=None, atprogram_path=None, configuration="Debug",
              device_sn=None, jobs=getenv("NUMBER_OF_PROCESSORS"),
              make_command=None, atprogram_command=None, dry_run=False):
    """Atprogram.

    This function can compile projects and write them to a device. It
    determines what to do based on the arguments it gets. Specify at least one
    of `project_path`, `make_command` or `atprogram_command`.

    In secure mode all commands will be returned in a list, they will not be
    executed.

    NOTE: Verification is known to return 23, also in command line.

    Keyword Arguments:
        project_path {str} -- Location where the project resides. If it ends in
            `.elf` the elf file will be used. If it ends in `Makefile` the
            Makefile will be used. Otherwise it should be a path to a folder
            which holds the `Debug` folder. (default: {None})
        device_name {str} -- Device name. E.g. atxmega128a1 or at32uc3a0256.
            (default: {"ATSAML11E16A"})
        verbose {int} -- Verbosity:
            - 0: Silent (default: {0})
            - 1: Info
            - 2: Debug
            - 3: List Commands
        clean {bool} -- Run make clean if True (default: {False})
        build {bool} -- Run make all if True (default: {True})
        erase {bool} -- Run atprogram chiperase if True (default: {True})
        program {bool} -- Run atprogram program if True (default: {True})
        verify {bool} -- Run atprogram verify if True (default: {False})
        tool {str} -- Tool name: avrdragon, avrispmk2, avrone, jtagice3,
            jtagicemkii, qt600, stk500, stk600, samice, edbg, medbg, nedbg,
            atmelice, pickit4, powerdebugger, megadfu or flip. (default:
            {"EDBG"})
        interface {str} -- Physical interface: aWire, debugWIRE, HVPP, HVSP,
            ISP, JTAG, PDI, UPDI, TPI or SWD. (default: {"SWD"})
        atmel_studio_path {[type]} -- Location where Atmel Studio is installed,
            ending in the folder named after the version, e.g. 7.0. (default:
            {path.join(getenv("programfiles(x86)"), "Atmel", "Studio", "7.0")})
        make_path {[type]} -- Location where `make.exe` is installed. (default:
            {path.join(getenv("programfiles(x86)"), "Atmel", "Studio", "7.0", "shellutils", "make.exe")})
        atprogram_path {[type]} -- Location where `atprogram.exe` is installed
            (default: {path.join(getenv("programfiles(x86)"), "Atmel", "Studio", "7.0", "atbackend", "atprogram.exe")})
        configuration {str} -- Which configuration to use. (default: {"Debug"})
        device_sn {str} -- The programmer/debugger serialnumber. Must be
            specified when more than one debugger is connected. (default:
            {None})
        jobs {int} -- How many jobs *make* should use(default:
            {getenv("NUMBER_OF_PROCESSORS")})
        make_command {str} -- Options to pass to make `[options] [target] ...`
            (default: {None})
        atprogram_command {str} -- Command(s) to pass to atprogram: `[options]
            <command> [arguments] [<command> [arguments] ...]` (default:
            {None})
        dry_run {bool} -- [description] (default: {False})

    Raises:
        ValueError -- Need to specify at least one of `project_path`,
            `make_command` or `atprogram_command`.

    Returns:
        int -- A non-zero return value indicates the subprocess call returned
            an error.

    """
    if SECURE_MODE:
        warn("In secure mode all commands will be returned in a list, they " +
             "will not be executed.")
    elf_mode = False
    makefile_mode = False
    makefile_path = None
    if dry_run or SECURE_MODE:
        dry_run_result = []
    if project_path is not None:
        elf_mode = path.splitext(project_path)[1] is ".elf"
        makefile_path, makefile = path.split(project_path)
        makefile_mode = makefile is "Makefile"
        if not makefile_mode:
            makefile_path = path.join(project_path, configuration)
    elif make_command is atprogram_command is None:
        raise ValueError(
            "Need to specify at least one of project_path, make_command or " +
            "atprogram_command.")
    else:
        # This is make_command mode or atprogram_command mode
        clean = build = erase = program = verify = False
    stdout = PIPE if verbose >= 0 else None
    stderr = STDOUT if verbose >= 1 else None
    if not elf_mode and (clean or build or make_command):
        make_path = make_path or path.join(
            atmel_studio_path, "shellutils", "make.exe")
    

def caller(command, program):
    if program is "make":
        args = ([make_path] + command.split())
        kwargs = dict(cwd=makefile_path, stdout=stdout, stderr=stderr)
    elif program is "atprogram":
        args = ([atprogram_path] + (verbose - 1) * ["-v"] + command.split())
    else:
        raise ValueError(
            "Program needs to be one of \"make\" or \"atprogram\".")
    kwargs = dict(stdout=stdout, stderr=stderr)
    if dry_run:
        dry_run_result.append(
            "".join(kwargs.get("cwd")) + "> " + " ".join(*args))
        if verbose:
            print(dry_run_result[-1])
        return 0
    res = run(args, **kwargs)
    if verbose:
        print(res.stdout.decode())
    return res.returncode

        if make_command:
            returncode = make_caller(make_command)
            if returncode or not atprogram_command:
                return returncode
        if clean:
            returncode = make_caller("clean")
            if returncode:
                return returncode
        if build:
            returncode = make_caller(f"all --jobs {jobs} --output-sync")
            if returncode:
                return returncode
    if not makefile_mode and (erase or program or verify or atprogram_command):
        atprogram_path = atprogram_path or path.join(
            atmel_studio_path, "atbackend", "atprogram.exe")

        elf_file_path = None if not (program or verify) else project_path if \
            elf_mode else path.join(project_path, configuration,
                                    path.basename(project_path)) + ".elf"
        atprogram_command = atprogram_command or \
            f"-t {tool} -i {interface} -d {device_name}" + \
            (device_sn is not None) * f" -s {device_sn}" + \
            erase * " chiperase" + \
            program * f" program -f {elf_file_path}" + \
            verify * f" verify -f {elf_file_path}" + \
            (verbose >= 3) * " info"
        returncode = atprogram_caller(atprogram_command)
        if returncode:
            atprogram_caller("exitcode")
            return returncode
    if dry_run or SECURE_MODE:
        return run.dry_run_result
    return 0
